In [15]:
import json
import requests

def search_by_taxonomy_code(taxonomy_code, city=None, state=None, limit=10):
    base_url = "https://npiregistry.cms.hhs.gov/api/"
    
    # Build query parameters
    params = {
        "version": "2.1",
        "limit": limit
    }
    
    # Use taxonomy_code parameter instead of taxonomy_description
    if taxonomy_code:
        params["taxonomy_code"] = taxonomy_code  # Using taxonomy_code parameter
    
    # Add location parameters if provided
    if city:
        params["city"] = city
    if state:
        params["state"] = state
    
    # Print the request URL for debugging
    print(f"Making request with params: {params}")
    
    # Make the API request
    try:
        response = requests.get(base_url, params=params)
        response.raise_for_status()
        
        result = response.json()
        # Check if there's an error in the response
        if "Errors" in result:
            print(f"API Error when searching for code '{taxonomy_code}':", result["Errors"])
            return None
        return result
    except requests.exceptions.RequestException as e:
        print(f"Request failed: {e}")
        return None

def display_provider_results(results, specialty_name):
    if not results or "results" not in results:
        print(f"No results found for {specialty_name}.")
        return

    doctors = results["results"]
    result_count = results.get("result_count", 0)
    
    if result_count == 0:
        print(f"No providers found for {specialty_name}.")
        return
    
    print(f"Found {result_count} {specialty_name} providers:")
    print("-" * 70)
    
    for i, doctor in enumerate(doctors, 1):
        # Basic information
        npi = doctor.get("number")
        basic = doctor.get("basic", {})
        name = basic.get("first_name", "") + " " + basic.get("last_name", "")
        
        if not name.strip():
            name = basic.get("organization_name", "Unknown Organization")
        
        # Address information
        addresses = doctor.get("addresses", [])
        location = "No address found"
        phone = ""
        
        for address in addresses:
            if address.get("address_purpose") == "LOCATION":
                street = address.get("address_1", "")
                city = address.get("city", "")
                state = address.get("state", "")
                zip_code = address.get("postal_code", "")
                phone = address.get("telephone_number", "")
                location = f"{street}, {city}, {state} {zip_code}"
                break
        
        # Taxonomy/specialty information
        taxonomies = doctor.get("taxonomies", [])
        specialties = []
        primary_taxonomy = ""
        
        for taxonomy in taxonomies:
            desc = taxonomy.get("desc")
            code = taxonomy.get("code")
            primary = taxonomy.get("primary")
            
            if desc:
                if primary:
                    primary_taxonomy = f"{desc} (Primary)"
                    specialties.insert(0, primary_taxonomy)
                else:
                    specialties.append(desc)
        
        specialty_str = ", ".join(specialties) if specialties else "No specialty listed"
        
        # Print formatted information
        print(f"{i}. {name} (NPI: {npi})")
        print(f"Address: {location}")
        if phone:
            print(f"Phone: {phone}")
        print(f"Specialty: {specialty_str}")
        print("-" * 70)

def search_dallas_mental_health():
    # City and state
    city = "DALLAS"
    state = "TX"
    
    # Taxonomy codes and their descriptions
    taxonomy_codes = {
        "2084P0804X": "Psychiatry & Neurology - Child & Adolescent Psychiatry",
        "261QM0855X": "Clinic/Center - Adolescent and Children Mental Health",
        "261QM0801X": "Clinic/Center - Mental Health (Including Community Mental Health Center)"
    }
    
    print(f"Searching for mental health providers in {city}, {state}...")
    print("=" * 80)
    
    # Search for each taxonomy code
    for code, description in taxonomy_codes.items():
        print(f"\nSearching for: {description} (Code: {code})")
        results = search_by_taxonomy_code(code, city=city, state=state, limit=10)
        if results:
            display_provider_results(results, description)
        else:
            print(f"No results or error when searching for {description}.")
        print("=" * 80)

# Run the search
if __name__ == "__main__":
    search_dallas_mental_health()

Searching for mental health providers in DALLAS, TX...

Searching for: Psychiatry & Neurology - Child & Adolescent Psychiatry (Code: 2084P0804X)
Making request with params: {'version': '2.1', 'limit': 10, 'taxonomy_code': '2084P0804X', 'city': 'DALLAS', 'state': 'TX'}
Found 10 Psychiatry & Neurology - Child & Adolescent Psychiatry providers:
----------------------------------------------------------------------
1. (EHH) ENTERPRISE HOME HEALTH SERVICES, LLC (NPI: 1518254648)
Address: 7557 RAMBLER RD, DALLAS, TX 752314142
Phone: 214-346-6050
Specialty: Home Health (Primary)
----------------------------------------------------------------------
2. -------------------------------------ABBY'S CDS FINANCIAL SERVICE LLC. (NPI: 1336555101)
Address: 4650 S HAMPTON RD # 125, DALLAS, TX 752321066
Phone: 469-222-1200
Specialty: Contractor (Primary)
----------------------------------------------------------------------
3. 1 CHIROPRACTIC (NPI: 1699258897)
Address: 15340 DALLAS PKWY STE 2740, DALLAS,